Before you turn in this homework, make sure everything runs as expected. First, restart the kernel (in the menubar, select Kernel $\rightarrow$ Restart) and then run all cells (in the menubar, select Cell $\rightarrow$ Run All).

Make sure you execute every single code cell, in order, filling with your solutions in any place that says `# YOUR CODE HERE`, and always DELETE the line that says:

```python
raise NotImplementedError()
```

The purpose of this line is to tell you if you forgot to answer a question (it will throw an error if the line is there)

**IMPORTANT:** DO NOT DELETE ANY CELL and do not change the title of the Notebook.

Fill your name below:

In [1]:
name = "Tianjiao Zhang"

# Coding Assignment: Rocket

The equations of motion for a rocket in purely vertical flight are given by

$$
\begin{align}
\frac{dh}{dt} &= v\\
(m_s+m_p) \frac{dv}{dt}& = -(m_s+m_p)g + \dot{m}_pv_e - \frac{1}{2}\rho v|v|AC_D
\end{align}
$$

$h$ is the altitude of the rocket

$m_s = 50kg$ is the weight of the rocket shell

$g = 9.81 \frac{m}{s^2}$

$\rho = 1.091 \frac{kg}{m^3}$ is the average air density (assumed constant throughout flight)

$A = \pi r^2$ is the maximum cross sectional area of the rocket, where $r = 0.5 m$

$v_e = 325 \frac{m}{s}$ is the exhaust speed

$C_D = 0.15 $ is the drag coefficient

$m_{po} = 100 kg$ at time $t = 0$ is the initial weight of the rocket propellant

The mass of the remaining propellant is given by:

$$m_p = m_{po} - \int^t_0 \dot{m}_p d\tau$$

where $\dot{m}_p$ is the time-varying burn rate given by the following figure:

Propellant Burn Rate

![burn rate](./figures/burn_rate.png)

that is,

$$
\begin{equation}
    \dot{m}_p \left( t \right) =
    \begin{cases}
        20 & \quad \text{if} \quad t < 5 \\
        0 & \quad \text{otherwise}
    \end{cases}
\end{equation}
$$

Using Euler's method with a time-step size of $\Delta t=0.1s$, create a Python script to calculate the altitude and velocity of the rocket from launch until crash down.

---

## Implement your solution

Implement your solution in this section.
You can use as many code cells as you want.

In [2]:
import numpy 
from matplotlib import pyplot
%matplotlib inline

In [3]:
pyplot.rcParams['font.family'] = 'serif'
pyplot.rcParams['font.size'] = 16

In [4]:
g = 9.81
ms = 50.0
mp0 = 100.0
dot_mp = 20.0
A = numpy.pi * 0.5**2
Ve = 325.0
density = 1.091
CD = 0.15



v0 = 0.0
h0 = 0.0

In [5]:
def rhs_rocket(u, g, dot_mp, Ve, M, density, A, CD):
    
    h, v = u
    rhs = numpy.array([v,
                       -g + dot_mp * Ve / M - 1 / (2*M) * density * v * abs(v) * A * CD])
    
    return rhs

In [6]:
def rhs2_rocket(u, g, ms, density, A, CD):
    
    h, v = u
    rhs = numpy.array([v,
                       -g - 1 / (2*ms) * density * v * abs(v) * A * CD])
    
    return rhs

In [7]:
T = 40.0
dt = 0.1
N = int(T / dt) + 1


M = numpy.empty(N)
for n in range(N - 1):
    if n < 50:
        M[n] = ms + mp0 - dot_mp * n * dt
    else:
        M[n] = ms

In [8]:
def euler_step(u, f, dt, *args):
    u_new = u + dt * f(u, *args)
    return u_new

In [9]:
u = numpy.empty((N, 2))
u[0] = numpy.array([h0, v0])


for n in range(N - 1):
    if n < 50:
        u[n + 1] = euler_step(u[n], rhs_rocket, dt, g, dot_mp, Ve, M[n], density, A, CD)
    else:
        u[n + 1] = euler_step(u[n], rhs2_rocket, dt, g, ms, density, A, CD)

In [10]:
h = u[:, 0]
v = u[:, 1]

In [11]:
idx_negative_v = numpy.where(v < 0.0)[0]
if len(idx_negative_v) == 0:
    idx_top_altitude = N - 1
    print('[H] Rocket has reached top altitude!')
else:
    idx_top_altitude = idx_negative_v[0]

In [12]:
idx_negative_h = numpy.where(h < 0.0)[0]
if len(idx_negative_h) == 0:
    idx_landing = N - 1
    print('[H] Rocket has landed!')
else:
    idx_landing = idx_negative_h[0]

---

## Assessment

In [13]:
# Import module to check answers.
import mooc

Answer questions in this section.

1. At time $t=3.2s$, what is the mass (in kg) of rocket propellant remaining in the rocket? 

_Instructions:_

* Store the value of the remaining propellant using a variable called `m_p`.
You can call the function `mooc.check('hw1_answer1', m_p)` to check you got the correct answer. The function will print `'Good job!'` if you got it right, `'Try again!'` otherwise.

In [15]:
m_p = 36
mooc.check('hw1_answer1', m_p)

[hw1_answer1] Good job!


2. What is the maximum speed of the rocket in $\frac{m}{s}$? At what time does this occur (in seconds)? What is the altitude at this time (in meters)?

_Instructions:_

* Store the maximum speed in the variable `vmax`, check your answer with `mooc.check('hw1_answer2', vmax)`.
* Store the time in `t_vmax`, check your answer with `mooc.check('hw1_answer3', t_vmax)`.
* Store the altitude in the variable `h_vmax`, check your answer with `mooc.check('hw1_answer4', h_vmax)`.

In [16]:
ind = numpy.argmax(v)
vmax = v[ind]
mooc.check('hw1_answer2', vmax)

[hw1_answer2] Good job!


In [17]:
t_vmax = 5
mooc.check('hw1_answer3', t_vmax)

[hw1_answer3] Good job!


In [19]:
h_vmax = h[50]
mooc.check('hw1_answer4', h_vmax)

[hw1_answer4] Good job!


3. What is the rocket's maximum altitude during flight (in meters)? At what time (in seconds) does this occur?

_Instructions:_

* Store the maximum altitude in the variable `hmax`, check your answer with `mooc.check('hw1_answer5', hmax)`.
* Store the time in the variable `t_hmax`, check your answer with `mooc.check('hw1_answer6', t_hmax)`.

In [20]:
hmax = h[idx_top_altitude]
mooc.check('hw1_answer5', hmax)

[hw1_answer5] Good job!


In [23]:
t_hmax  = idx_top_altitude * dt
mooc.check('hw1_answer6', t_hmax)

[hw1_answer6] Good job!


4. At what time (in seconds) does the rocket impact the ground? What is the velocity of the rocket (in $\frac{m}{s}$) at time of impact?

_Instructions:_

* Store the time of impact in the variable `t_impact`, check your answer with `mooc.check('hw1_answer7', t_impact)`.
* Store the impact velocity in the variable `v_impact`, check your answer with `mooc.check('hw1_answer8', v_impace)`.

In [96]:
T = 40.0
dt = 0.01
N = int(T / dt) + 1


M = numpy.empty(N)
for n in range(N-1):
    if n < 500:
        M[n] = ms + mp0 - dot_mp * n * dt
    else:
        M[n] = ms
        
        
u = numpy.empty((N, 2))
u[0] = numpy.array([h0, v0])


for n in range(N-1):
    if n < 500:
        u[n + 1] = euler_step(u[n], rhs_rocket, dt, g, dot_mp, Ve, M[n], density, A, CD)
    else:
        u[n + 1] = euler_step(u[n], rhs2_rocket, dt, g, ms, density, A, CD)
        
        
h = u[:, 0]
v = u[:, 1]


idx_negative_h = numpy.where(h < 0.0)[0]
if len(idx_negative_h) == 0:
    idx_landing = N - 1
    print('[H] Rocket has landed!')
else:
    idx_landing = idx_negative_h[0]
        


print(idx_landing)

t_impact = 37.07

mooc.check('hw1_answer7', t_impact)

3733
[hw1_answer7] Good job!


In [33]:
v_impact = v[idx_landing - 1]
mooc.check('hw1_answer8', v_impact)

[hw1_answer8] Good job!


---

## Derivation of the rocket equations

In case you are kind of confused about the rocket equations, here we show how to get to them. 

Newton's second law states that the acceleration of the vehicle times its mass is equal to all the forces acting on it. Therefore,

\begin{equation}
(m_s + m_p)\frac{d\bf{v}}{dt}=\sum {\bf F}.
\end{equation}
In the above formula we have assumed that the propellant inside the rocket and the rocket move at the same velocity (in other words, their relative velocity is negligible). 

Two of the external forces acting on the rocket are,

\begin{align}
{\bf F}_g&= (m_s+m_p)\bf{g} \quad (\rm{Gravity}),\\
{\bf F}_d&= - \frac{1}{2} \rho_a \mathbf{v} |\mathbf{v}| A  C_D \quad (\rm{Drag}).
\end{align}

We also need to consider the force resulting from the ejection of the propellant. During an interval $dt$, the engine of the rocket ejects downwards a mass of propellant given by $\dot m_p dt$. Relative to the rocket, the speed of the ejected burning gas is assumed constant and equal to $v_e$ (the exhaust speed). The momentum variation induced on the exhaust gas by the engine during that interval is therefore, $d{\bf p}_{gas} = \dot m_p {\bf v}_e dt$. Again using Newton's second law we conclude that the force applied by the rocket on the gas is,

\begin{align}
{\bf F}_{rocket\rightarrow gas} = \frac{d{\bf p}_{gas}}{dt} = \dot m_p {\bf v}_e
\end{align}

Using Newton's third law (|action| = |reaction|), the force exerted by the exhaust gas on the rocket is then,

\begin{align}
{\bf F}_{gas\rightarrow rocket} = -{\bf F}_{rocket\rightarrow gas} = -\dot m_p {\bf v}_e
\end{align}

If we collect all the forces acting on the rocket we finally have:

\begin{align}
(m_s + m_p)\frac{d\bf{v}}{dt}=(m_s+m_p){\bf g}- \frac{1}{2} \rho_a \mathbf{v} |v| A  C_D -\dot m_p {\bf v}_e
\end{align}



---

###### The cell below loads the style of the notebook.

In [ ]:
from IPython.core.display import HTML
css_file = './styles/numericalmoocstyle.css'
HTML(open(css_file, 'r').read())